In [42]:
import pandas as pd

#Importo le informazioni riguardanti le qualità delle tasks 
tasks = pd.read_csv("../Dataset 2.0/Tasks.csv")
tasks = tasks.drop(0, axis = 0)
tasks = tasks.drop("Deadline", axis = 1)


#Importo i coefficienti di regressione di ogni agente
coeffs = pd.read_csv("../Coefficienti di Regressione/coeffs.csv")

decisions = pd.read_csv("../Dataset 2.0/Decisions.csv")

#Selezionare il Session ID ed il Round voluti
#Assicurarsi di selezionare lo stesso round in "RegressionModel.py" e in "OptimizationModel.py"
decisions = decisions[decisions["Session ID"] == "e7e52776-2750-4cde-a5da-093ccb8feaf9"]
decisions = decisions[decisions["Round"] == 1]
decisions = decisions.sort_values("Worker Agent ID")
decisions

,ID,Session ID,Round,Worker Agent ID,Worker Agent Reputation
0,1,e7e52776-2750-4cde-a5da-093ccb8feaf9,1,1,0.5
7,15,e7e52776-2750-4cde-a5da-093ccb8feaf9,1,2,0.5
2,5,e7e52776-2750-4cde-a5da-093ccb8feaf9,1,3,0.5
5,11,e7e52776-2750-4cde-a5da-093ccb8feaf9,1,4,0.5
3,7,e7e52776-2750-4cde-a5da-093ccb8feaf9,1,5,0.5
1,3,e7e52776-2750-4cde-a5da-093ccb8feaf9,1,6,0.5
6,13,e7e52776-2750-4cde-a5da-093ccb8feaf9,1,7,0.5
4,9,e7e52776-2750-4cde-a5da-093ccb8feaf9,1,8,0.5
8,17,e7e52776-2750-4cde-a5da-093ccb8feaf9,1,9,0.5
9,19,e7e52776-2750-4cde-a5da-093ccb8feaf9,1,10,0.5


In [44]:
from mip import *

m = Model(sense=MAXIMIZE, solver_name=CBC)

#DATI

#Identifier del Round
dID = list(decisions["ID"])


#Peso dell'Identifier del round
wdID = coeffs.iloc[0].values.tolist()


#Identifier del Worker Agent
WAID = list(decisions["Worker Agent ID"])

#Peso dell'Identifier del WA
wWAID = coeffs.iloc[91].values.tolist()

if (WAID[9] == 10):
    wWAID = wWAID[0:10]

else:
    wWAID = wWAID[10:20]


#Value di ogni task
Value = []
for i in tasks["Value"]:
    Value.append(int(i))
    
if (WAID[9] == 10):
    Value = Value[0:10]

else:
    Value = Value[10:20]
    

#Peso del Value (coeffs 1, 4, 7, ..., 88)
wValue = []

j = 1
for k in range(30):
    wValue.append(coeffs.iloc[j].values.tolist())
    j = j + 3
    
if (WAID[9] == 10):
    wValue = wValue[0:10]

else:
    wValue = wValue[10:20]


#Complessità di ogni task
Difficulty = []
for i in tasks["Difficulty"]:
    Difficulty.append(float(i))
    
if (WAID[9] == 10):
    Difficulty = Difficulty[0:10]

else:
    Difficulty = Difficulty[10:20]


#Peso della Complessità (coeffs 2, 5, 8, ..., 89)
wDifficulty = []

j = 2
for k in range(30):
    wDifficulty.append(coeffs.iloc[j].values.tolist())
    j = j + 3
    
if (WAID[9] == 10):
    wDifficulty = wDifficulty[0:10]

else:
    wDifficulty = wDifficulty[10:20]


#Stamina consumata da ogni task
Effort = []
for i in tasks["Effort Required"]:
    Effort.append(int(i))
  
if (WAID[9] == 10):
    Effort = Effort[0:10]

else:
    Effort = Effort[10:20]

#Peso della Stamina consumata (coeffs 3, 6, 9, ..., 90)
wEffort = []

j = 3
for k in range(30):
    wEffort.append(coeffs.iloc[j].values.tolist())
    j = j + 3

if (WAID[9] == 10):
    wEffort = wEffort[0:10]

else:
    wEffort = wEffort[10:20]
    
#Reputazione del WA prima del round
Rep = list(decisions["Worker Agent Reputation"])


#Peso della reputation
wRep = coeffs.iloc[92].values.tolist()

if (WAID[9] == 10):
    wRep = wRep[0:10]

else:
    wRep = wRep[10:20]

#creo i range utili per la creazione di variabili
(A, T) = (range(len(WAID)), range(len(Value)))


#VARIABILI

#x[a, t] vale 1 se la task t è assegnata al WA a, 0 altrimenti
x = [[m.add_var(var_type=BINARY) for t in T] for a in A]

#y(a) contiene la reputation del WA a fine round
y = [m.add_var(var_type=CONTINUOUS) for a in A]


#VINCOLI

#Vincolo che impone che ogni task venga assegnata ad uno ed un solo WA
for t in T:
    m = xsum(x[a][t] for a in A) == 1


#OBIETTIVO
#Massimizzare la sommatoria delle reputation dei worker agent
m.objective = maximize(xsum(dID[a]*wdID[a] for a in A) + xsum(wValue[a][t]*Value[t]*x[a][t] + wDifficulty[a][t]*Difficulty[t]*x[a][t] + wEffort[a][t]*Effort[t]*x[a][t] for t in T for a in A) + xsum(wRep[a]*Rep[a] + wWAID[a]*WAID[a] for a in A))

m.write("model.lp")
m.optimize()

print(f"Totale: {m.objective_value}")

AttributeError: 'LinExpr' object has no attribute 'objective'